In [2]:
# Импортируем библиотеки

import requests
import pandas as pd
from bs4 import BeautifulSoup
import time

from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys

from tqdm import tqdm

import csv

import json
from datetime import datetime

import pandas as pd

In [3]:
# Запускаем WEB-драйвер

driver=webdriver.Chrome()

driver.get('https://incrussia.ru/')

In [4]:
# Кликаем на кнопку «загрузить еще» 21 раз

for i in tqdm(range(21)):
      
    element = driver.find_element("xpath", "//div[@class='loader-btn']")
    
    driver.execute_script("arguments[0].click();", element)
    
    time.sleep(10)

100%|██████████| 20/20 [03:21<00:00, 10.08s/it]


Документация BS4 https://www.crummy.com/software/BeautifulSoup/bs4/doc.ru/bs4ru.html

In [8]:
# Испольщуем метод BeautifulSoup, который представляет HTML-документ в виде вложенной структуры данных

soup = BeautifulSoup(driver.page_source, "html.parser")

In [9]:
# Получаем названия всех <a>-тегов из документа

tegs_a = soup.find_all('a')

classes_a = []
statistics_classes_a = []

for a in tqdm(tegs_a):
    class_name = a.get('class')
    if class_name is None:
        continue
    
    if ' '.join(class_name) not in classes_a:
        classes_a.append(' '.join(class_name))
        
for class_name in classes_a:
    statistics_classes_a.append((class_name, len(soup.find_all('a', class_=class_name))))

statistics_classes_a.sort(key = lambda x: x[1], reverse=True)

for name, cnt in statistics_classes_a:
    print(name, cnt)

100%|██████████| 2321/2321 [00:00<00:00, 716017.92it/s]


journal_line_item_category 722
journal_line_item_name text-link article-link 362
backdrop-link 362
journal_line_item_name text-link 360
news-item_name 79
news-item_name text-link 15
search_found_item_name text-link 5
mobile-order-2 3
soc m-vk 2
soc m-tw 2
soc m-ig 2
april-header-logo 2
journal-title_item_link 2
mobile-order-3 2
mobile-order-1 2
header_logo progress_logo 1
april-post 1
logo 1
journal-title_item_name 1
mobile-order-4 1
mobile-order-5 1
mobile-order-6 1
vk 1
tw 1
od 1
tme 1
active 1
testBanner-link 1
close 1


In [58]:
# Оставляем те <a>-тегов, в которых содержатся статьи

correct_a_classes = ["journal_line_item_name text-link article-link",
                    "journal_line_item_name text-link",
                    "news-item_name",
                    "news-item_name text-link"]

In [70]:
# Записываем в csv-файл ссылки и заголовки статей, которые мы собрали по <a>-тегов

# Создаем csv-файл
columns = ['link', 'title']

with open('incrussia.csv', 'w', encoding='utf-16', newline='') as file:
    writer = csv.writer(file, delimiter=';')
    writer.writerow(columns)
    
    
# Записываем в csv-файл
for a_class in correct_a_classes:
    soup_a = soup.find_all('a', class_=a_class)
    for one in soup_a:

        link = one['href']
        title = one.text.replace('\xa0', ' ')    
        row = [link, title]    

        with open('incrussia.csv', 'a', encoding='utf-16', newline='') as file:
            writer = csv.writer(file, delimiter=';')
            writer.writerow(row)

In [84]:

df = pd.read_csv('incrussia.csv', delimiter=';', encoding='utf-16')

In [88]:
df.drop_duplicates(inplace=True)

In [89]:
df

,link,title
0,https://incrussia.ru/brandview/two-years-suppo...,Два года поддержки: как поменялась платформа М...
1,https://incrussia.ru/understand/rynochnaya-mat...,Рыночная математика: почему стартапы ошибаются...
2,https://incrussia.ru/understand/antistress-v-o...,Антистресс в офисе: как работодатели берегут п...
3,https://incrussia.ru/understand/anglijskij-dly...,Английский для бизнеса: как и зачем отслеживат...
4,https://incrussia.ru/switch/peredacha-otvetstv...,Передача ответственности: как перейти на аутсо...
...,...,...
741,https://incrussia.ru/news/parlament-es-prinyal...,Парламент ЕС принял первый в мире закон о регу...
742,https://incrussia.ru/news/issledovanie-alkogol...,Исследование: алкоголь и наркотики влияют на р...
743,https://incrussia.ru/news/lucid-motors-ne-nach...,Lucid Motors не начала производство электровне...
744,https://incrussia.ru/news/porsche-i-chempion-f...,Porsche и чемпион «Формулы-1» инвестировали $2...


In [262]:
def get_text(url):

    # Отправляем GET-запрос и получаем содержимое страницы
    response = requests.get(url)

    # Создаем объект BeautifulSoup для парсинга HTML
    soup = BeautifulSoup(response.content, 'html.parser')
    
    # Достаем дату
    all_script = soup.find_all('script', type='application/ld+json')

    date_iso = json.loads(list(filter(lambda x: 'datePublished' in x.text, all_script))[0].text)

    date = datetime.fromisoformat(date_iso['datePublished']).strftime("%d.%m.%Y")
    

    # Находим элементы с текстом статьи (обычно это теги <p> или <div>)
    article_text = soup.find_all('p')
    
    # Убираем параграфы с указанием даты, рубрики, авторов текста и фото
    stop_info = ['•', 'Текст:', 'Фото:']

    for stop_one in stop_info:
        article_text = list(filter(lambda x: stop_one not in x.text, article_text))

    # Извлекаем текст из найденных элементов
    text = ''
    for paragraph in article_text[0:-3]:

        text += paragraph.text.replace(u"\xa0", u" ") + ' '

    text = text.replace('\n', ' ').replace('  ', ' ')
    # Выводим текст статьи
    return [url, text, date]

Защитил следующую ячейку, чтобы случайно не выполнить ее повторно

In [265]:
# Записываем в csv-файл ссылки, тексты и даты публикации статей

for url in tqdm(df.link.to_list()):
    
    row = get_text(url)

    with open('incrussia_text.csv', 'a', encoding='utf-16', newline='') as file:
            writer = csv.writer(file, delimiter=';')
            writer.writerow(row)
            
    time.sleep(7)

100%|██████████| 746/746 [1:34:51<00:00,  7.63s/it]


In [269]:
df_content = pd.read_csv('incrussia_text.csv', delimiter=';', encoding='utf-16')

In [270]:
df_content.drop_duplicates(inplace=True)

In [274]:
df_content

,link,text,date
0,https://incrussia.ru/brandview/two-years-suppo...,пользователей. пользователей добавляется ежеме...,13.03.2024
1,https://incrussia.ru/understand/rynochnaya-mat...,Рыночная математика: почему стартапы ошибаются...,29.02.2024
2,https://incrussia.ru/understand/antistress-v-o...,Антистресс в офисе: как работодатели берегут п...,29.02.2024
3,https://incrussia.ru/understand/anglijskij-dly...,Английский для бизнеса: как и зачем отслеживат...,26.02.2024
4,https://incrussia.ru/switch/peredacha-otvetstv...,Передача ответственности: как перейти на аутсо...,26.02.2024
...,...,...,...
741,https://incrussia.ru/news/parlament-es-prinyal...,Парламент Европейского Союза утвердил первый в...,14.03.2024
742,https://incrussia.ru/news/issledovanie-alkogol...,"Мозг воспринимает пищу как вознаграждение, ког...",14.03.2024
743,https://incrussia.ru/news/lucid-motors-ne-nach...,Lucid Motors рискует потерять торговую марку д...,14.03.2024
744,https://incrussia.ru/news/porsche-i-chempion-f...,Компания-разработчик программного обеспечения ...,13.03.2024


In [275]:
result = df.merge(df_content, on='link')
result

,link,title,text,date
0,https://incrussia.ru/brandview/two-years-suppo...,Два года поддержки: как поменялась платформа М...,пользователей. пользователей добавляется ежеме...,13.03.2024
1,https://incrussia.ru/understand/rynochnaya-mat...,Рыночная математика: почему стартапы ошибаются...,Рыночная математика: почему стартапы ошибаются...,29.02.2024
2,https://incrussia.ru/understand/antistress-v-o...,Антистресс в офисе: как работодатели берегут п...,Антистресс в офисе: как работодатели берегут п...,29.02.2024
3,https://incrussia.ru/understand/anglijskij-dly...,Английский для бизнеса: как и зачем отслеживат...,Английский для бизнеса: как и зачем отслеживат...,26.02.2024
4,https://incrussia.ru/switch/peredacha-otvetstv...,Передача ответственности: как перейти на аутсо...,Передача ответственности: как перейти на аутсо...,26.02.2024
...,...,...,...,...
741,https://incrussia.ru/news/parlament-es-prinyal...,Парламент ЕС принял первый в мире закон о регу...,Парламент Европейского Союза утвердил первый в...,14.03.2024
742,https://incrussia.ru/news/issledovanie-alkogol...,Исследование: алкоголь и наркотики влияют на р...,"Мозг воспринимает пищу как вознаграждение, ког...",14.03.2024
743,https://incrussia.ru/news/lucid-motors-ne-nach...,Lucid Motors не начала производство электровне...,Lucid Motors рискует потерять торговую марку д...,14.03.2024
744,https://incrussia.ru/news/porsche-i-chempion-f...,Porsche и чемпион «Формулы-1» инвестировали $2...,Компания-разработчик программного обеспечения ...,13.03.2024


In [277]:
result.to_csv('incrussia_2021_2024.csv', index=False, encoding='utf-16', sep='\t')